In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#인셉션 네트워크 
#filter 크기가 다른 cnn 나눠 진행후 채널방향으로 연결 > 서로 다른 특징 찾기 용이
#연산 수가 훨씬 많이 필요한 상황에서 1x1 conv를 통해 channel 을 줄였다가 다음 cnn에서 
#확장 > 연산수 줄여줌

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

batch_size= 1
learning_rate = 0.0002
epoch = 100



In [ ]:
#폴더별로 라벨이 구분되어 있을 때 사용
trainset = dset.ImageFolder('/content/drive/My Drive/data',
                               transform = transforms.Compose([
                                  transforms.Resize(224), 
                                  transforms.ToTensor() ,
                                  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),                           
                               ])
                         )

trainloader = DataLoader(trainset, batch_size=1, shuffle=True, num_workers=2)
print(len(trainloader[0]),len(trainloader[1])) 

200


In [ ]:
# conv 1X1 
def conv_1(in_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim,1,1),
      nn.ReLU(),
  )
  return model 

# conv 1X1 1X3 
def conv_1_3(in_dim, mid_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim,mid_dim,1,1),
      nn.ReLU(),
      nn.Conv2d(mid_dim,out_dim,3,1,1),
      nn.ReLU(),
  )
  return model

#conv 1X1 1X5
def conv_1_5(in_dim,mid_dim,out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim,mid_dim,1,1),
      nn.ReLU(),
      nn.Conv2d(mid_dim,out_dim,5,1,2),
      nn.ReLU(),
  )
  return model

#maxpooling 후 conv 1X1
def max_3_1(in_dim,out_dim):
  model = nn.Sequential(
      nn.MaxPool2d(3,1,1),
      nn.Conv2d(in_dim,out_dim,1,1),
      nn.ReLU(),
  )
  return model


In [ ]:
class inception_module(nn.Module):

  def __init__(self, in_dim, out_dim_1, mid_dim_3, out_dim_3, mid_dim_5, out_dim_5,pool):
    super(inception_module,self).__init__()

    self.conv_1 = conv_1(in_dim,out_dim_1)
    self.conv_1_3 = conv_1_3(in_dim,mid_dim_3, out_dim_3)
    self.conv_1_5 = conv_1_5(in_dim, mid_dim_5, out_dim_5)
    self.max_3_1 = max_3_1(in_dim,pool)

  def forward(self, x):
    out_1 = self.conv_1(x)
    out_2 = self.conv_1_3(x)
    out_3 = self.conv_1_5(x)
    out_4 = self.max_3_1(x)
    output = torch.cat([out_1, out_2, out_3, out_4],1)

    return output


In [ ]:
class GoogLeNet(nn.Module):
  def __init__(self, base_dim, num_classes = 2):
    super(GoogLeNet, self).__init__()
    self.layer_1 = nn.Sequential(
        #in out kernel stride padding
        nn.Conv2d(3,base_dim,7,2,3),
        #kernel  stride padding
        nn.MaxPool2d(3,2,1),
        nn.Conv2d(base_dim,base_dim*3,3,1,1), 
        nn.MaxPool2d(3,2,1),

    )

    self.layer_2 = nn.Sequential(
        #in_dim, out_dim_1, mid_dim_3, out_dim_3, mid_dim_5, out_dim_5,pool
        inception_module(base_dim*3,64,96,128,16,32,32),
        inception_module(base_dim*4,128,128,192,32,96,64),
        nn.MaxPool2d(3,2,1)
    )

    self.layer_3 = nn.Sequential(
        inception_module(480,192,96,208,16,48,64),
        inception_module(512,160,112,224,24,64,64),
        inception_module(512,128,128,256,24,64,64),
        inception_module(512,112,144,288,32,64,64),
        inception_module(528,256,160,320,32,128,128),
        nn.MaxPool2d(3,2,1),
    )
    self.layer_4 = nn.Sequential(
            inception_module(832,256,160,320,32,128,128),
            inception_module(832,384,192,384,48,128,128), 
            nn.AvgPool2d(7,1),
        )
    
    self.layer_5 = nn.Dropout2d(0.4)
    self.fc_layer = nn.Linear(1024,1)  
  def forward(self, x):
    out = self.layer_1(x)
    out = self.layer_2(out)
    out = self.layer_3(out)
    out = self.layer_4(out)
    out = self.layer_5(out)

    out = out.view(batch_size,-1)
    out = self.fc_layer(out).view(-1)
    return out

model = GoogLeNet(base_dim = 64).cuda()

for i in model.named_children():
  print(i)

In [ ]:
loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(),lr= learning_rate)

for i in range(epoch):
  for img, label in trainloader: 
    
    optimizer.zero_grad()
    output = model(img.cuda())
    loss = loss_function(output, label.cuda().float())
    loss.backward()
    optimizer.step()

  if i % 10 == 0:
    print(loss)

tensor(0.6898, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.6914, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.6991, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.6946, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.6874, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.6954, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.6889, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.6960, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.6738, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.6736, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
